# Acoustic non-linear wave-equation operator

## Acoustic isotropic wave equation and stepping rules

In this notebook we show how to derive and implement acoustic isotropic wave-equation operators with simple stepping rules. We start by considering the acoustic isotropic wave equation that can be written as follows:

\begin{equation}
\left[\frac{1}{\mathbf{v}^2(\mathbf{x})}\frac{\partial^2}{\partial t^2} - \nabla^2 \right]\mathbf{p}(\mathbf{x},t) = \mathbf{f}(\mathbf{x},t),
\end{equation}

where $\mathbf{p}$ represents the pressure field, $\mathbf{f}$ is the forcing term, and $\mathbf{v}$ is the acoustic velocity. Let's now discretize in time this equation and use a second-order scheme and drop the dependency over the spatial coordinate vector $\mathbf{x}$. The previous equation now becomes:

\begin{equation}
\frac{\mathbf{p}(t_{i+1}) -2\mathbf{p}(t_{i}) +\mathbf{p}(t_{i-1})}{\Delta t^2 \mathbf{v}^2} - \nabla^2\mathbf{p}(t_i) = \mathbf{f}(t_i),
\end{equation}

in which the variable $i$ represents index of time sample in the discrete axis with $i \in  [0,\dots,N_t]$. Assuming that $\mathbf{p}(t) = 0$ for $t \leq 0$, we can write the second-order time derivative as a lower triangular matrix: 

\begin{equation}
\mathbf{D}^2_t = \frac{1}{\Delta t^{2}}\begin{bmatrix} 1 & 0 & 0 & \cdots & 0 & 0 & 0 \\
                                                 -2 & 1 & 0 & \cdots & 0 & 0 & 0 \\
                                                 1 & -2 & 1 & \cdots & 0 & 0 & 0 \\
                                                 \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
                                                 0 & 0 & 0 & \cdots & 1 & 0 & 0 \\
                                                 0 & 0 & 0 & \cdots & -2 & 1 & 0 \\
                                                 0 & 0 & 0 & \cdots & 1 & -2 & 1
             \end{bmatrix},
\end{equation}

where $\Delta t$ represents the time sampling, giving rise to the following time-stepping rule:

\begin{equation}
\mathbf{p}(t_{i}) =\Delta t^2 \mathbf{v}^2\mathbf{f}(t_{i-1}) + [\Delta t^2 \mathbf{v}^2\nabla^2 +2] \mathbf{p}(t_{i-1}) - \mathbf{p}(t_{i-2}),
\end{equation}

whose syntax can be simplified by introducing $\mathbf{A}=\Delta t^2 \mathbf{v}^2$ and $\mathbf{B}=\mathbf{A}\nabla^2+2$. Therefore, we can rewrite the previous stepping rule as follows:

\begin{equation}
\mathbf{p}(t_{i}) =\mathbf{A}\mathbf{f}(t_{i-1}) + \mathbf{B} \mathbf{p}(t_{i-1}) - \mathbf{p}(t_{i-2}).
\end{equation}

Alomomin (2013) states the following: "Although the previous equation describes the forward time-stepping correctly, its
adjoint is not a recursive operator.". Hence, in their derivation of the adjoint stepping rule they define the $\mathbf{q}(t_{i})=\mathbf{A}\mathbf{f}(t_{i})$, which changes the forward stepping rule as follows:

\begin{equation}
\mathbf{p}(t_{i}) =\mathbf{q}(t_{i-1}) + \mathbf{B} \mathbf{p}(t_{i-1}) - \mathbf{p}(t_{i-2}),
\end{equation}

which allows them to write the following adjoint stepping rule:

\begin{equation}
\mathbf{q}(t_{i}) =\mathbf{p}(t_{i+1}) + \mathbf{B}^{*} \mathbf{q}(t_{i+1}) - \mathbf{q}(t_{i+2}),
\end{equation}

where adjoint operator of $\mathbf{B}$ is given by the following expression:

\begin{equation}
\mathbf{B}^{*} = \nabla^{2*}\mathbf{A}^{*} = \nabla^{2*}\mathbf{A},
\end{equation}

which shows that it is necessary to apply the operator $\mathbf{A}$ before computing the adjoint Laplacian operator $\nabla^{2*}$. Even though this simple change of operation order seems a small detail, it actually results in a more complex adjoint operator and possibly in a less efficient implementation. In the next section we show how to avoid this issue and implement a simpler stepping rule that avoids the usage of the operator $\mathbf{B}^{*}$ altogether.

## A different adjoint stepping rule

Let's go back to the time-discrete version of the wave-equation introduced in the previous section. Using the operator $\mathbf{D}^2_t$, we can write:

\begin{equation}
\left[\frac{1}{\mathbf{v}^2}\mathbf{D}^2_t - \nabla^2\right]\mathbf{p} = \mathbf{F} \mathbf{p} = \mathbf{f},
\end{equation}

where $\mathbf{p}=[\mathbf{p}(t_0),\dots,\mathbf{p}(t_{N_t})]^{*}$ and $\mathbf{f}=[\mathbf{f}(t_0),\dots,\mathbf{f}(t_{N_t})]^{*}$ . The previous section showed that this linear system can be solved by Gaussian substitution to find $\mathbf{p} = \mathbf{F}^{-1}\mathbf{f}$. To find the adjoint $\mathbf{F}^{*}$ and its inverse operator let's write the following expression:

\begin{equation}
 \mathbf{F}^{*} \mathbf{f} = \left[\frac{1}{\mathbf{v}^2}\mathbf{D}^2_t - \nabla^2\right]^{*} \mathbf{f} = \left[\frac{1}{\mathbf{v}^2}\mathbf{D}^{2*}_t - \nabla^{2*}\right] \mathbf{f} = \mathbf{p},
\end{equation}

where we used the fact that $\frac{1}{\mathbf{v}^2}$ commutes with the derivative operator $\mathbf{D}^{2*}_t$. This equation shows that the only difference from the forward equation is given by the way we solve it, since in the adjoint equation we have a upper triangular system, and the presence of $\nabla^{2*}$ as opposed to $\nabla^{2}$, which can be removed if $\nabla^{2*}=\nabla^{2}$
Again, if we solve the adjoint wave equation by Gaussian substitution, we can write the adjoint stepping rule as follows:

\begin{equation}
\mathbf{f}(t_{i}) =\mathbf{A}\mathbf{p}(t_{i+1}) + \mathbf{B} \mathbf{f}(t_{i+1}) - \mathbf{f}(t_{i+2}),
\end{equation}

where we assume $\nabla^{2}$ to be self-adjoint, assumption often fulfilled unless particular boundary conditions are employed (e.g., free-surface boundary). Beside the change in time indices, this stepping rule is computationally equivalent to the one present in the forward wave-equation operator. Let's test if this derivation passes the dot-product test. 

## Numerical test

Let's start by importing the necessary modules

In [1]:
import sys
sys.path.append("/Users/ettorebiondi/PycharmProjects/python-solver/GenericSolver/python")
import pyVector as Vec
import pyOperator as Op
import numpy as np

WARNING! DATAPATH not found. The folder /tmp will be used to write binary files


Now, lets define the wave-equation operator that makes use of the stepping rules described in the previous section.

In [2]:
class AcousticWaveEquation(Op.Operator):
	def __init__(self,wavefield,vel,dt,ds):
		self.setDomainRange(wavefield,wavefield)
		self.vel = vel
		self.dt = dt
		self.dt2 = dt*dt
		self.ds2_inv = 1.0/(ds*ds)
		self.vel2dt = self.vel.getNdArray()*self.vel.getNdArray()*self.dt2

	def forward(self,add,model,data):
		self.checkDomainRange(model,data)
		if not add:
			data.zero()
		m_arr = model.clone().getNdArray()
		# Scaling input wavefield
		m_arr *= self.vel2dt
        # Getting number of samples per axis
		data_tmp = data.clone().zero()
		d_arr = data_tmp.getNdArray()
		nt = m_arr.shape[0]
		nx = m_arr.shape[1]
		nz = m_arr.shape[2]
        # Imposing spatial boundary conditions
		m_arr[:,0,:]=0.0
		m_arr[:,-1,:]=0.0
		m_arr[:,:,0]=0.0
		m_arr[:,:,-1]=0.0
        # Stepping 
		for it in range(nt):
			if it > 1:
				for idx in range(1,nx-1):
					for idz in range(1,nz-1):
						d_arr[it,idx,idz] += (d_arr[it-1,idx,idz-1]+d_arr[it-1,idx,idz+1]-4.0*d_arr[it-1,idx,idz]+d_arr[it-1,idx-1,idz]+d_arr[it-1,idx+1,idz])*self.ds2_inv*self.vel2dt[idx,idz] \
                                              + 2.0*d_arr[it-1,idx,idz] - d_arr[it-2,idx,idz] + m_arr[it-1,idx,idz]
			elif it == 1:
				for idx in range(1,nx-1):
					for idz in range(1,nz-1):
						d_arr[it,idx,idz] += (d_arr[it-1,idx,idz-1]+d_arr[it-1,idx,idz+1]-4.0*d_arr[it-1,idx,idz]+d_arr[it-1,idx-1,idz]+d_arr[it-1,idx+1,idz])*self.ds2_inv*self.vel2dt[idx,idz] \
                                              + 2.0*d_arr[it-1,idx,idz] + m_arr[it-1,idx,idz]
		data.scaleAdd(data_tmp)


	def adjoint(self,add,model,data):
		self.checkDomainRange(model,data)
		if not add:
			model.zero()
		d_arr = data.clone().getNdArray()
		model_tmp = model.clone().zero()
		m_arr = model_tmp.getNdArray()
        # Scaling input wavefield
		d_arr *= self.vel2dt
        # Getting number of samples per axis
		nt = m_arr.shape[0]
		nx = m_arr.shape[1]
		nz = m_arr.shape[2]
        # Imposing spatial boundary conditions
		d_arr[:,0,:]=0.0
		d_arr[:,-1,:]=0.0
		d_arr[:,:,0]=0.0
		d_arr[:,:,-1]=0.0
        # Stepping 
		for it in range(nt,-1,-1):
			if it < nt-2:
				for idx in range(1,nx-1):
					for idz in range(1,nz-1):
						m_arr[it,idx,idz] += (m_arr[it+1,idx,idz-1]+m_arr[it+1,idx,idz+1]-4.0*m_arr[it+1,idx,idz]+m_arr[it+1,idx-1,idz]+m_arr[it+1,idx+1,idz])*self.ds2_inv*self.vel2dt[idx,idz] \
                                              + 2.0*m_arr[it+1,idx,idz] - m_arr[it+2,idx,idz] + d_arr[it+1,idx,idz]
			elif it == nt-2:
				for idx in range(1,nx-1):
					for idz in range(1,nz-1):
						m_arr[it,idx,idz] += (m_arr[it+1,idx,idz-1]+m_arr[it+1,idx,idz+1]-4.0*m_arr[it+1,idx,idz]+m_arr[it+1,idx-1,idz]+m_arr[it+1,idx+1,idz])*self.ds2_inv*self.vel2dt[idx,idz] \
                                              + 2.0*m_arr[it+1,idx,idz] + d_arr[it+1,idx,idz]
		model.scaleAdd(model_tmp)

Now, let's instantiate an acoustic wave-equation operator and test its adjointness.

In [5]:
nx=150
nz=210
nt=60
wav = Vec.vectorIC(np.zeros((nt,nx,nz)))
vel = Vec.vectorIC(np.zeros((nx,nz)))
vel.set(2.0)
vel.getNdArray()[:,20:] = 2.5
vel.getNdArray()[:,:] += np.random.rand(nx,nz)*0.1
ds = 0.02
dt = 0.001
print("Testing wave-equation operator")
prop = AcousticWaveEquation(wav,vel,dt,ds)
prop.dotTest(True)

Testing wave-equation operator
Dot-product test of forward and adjoint operators
-------------------------------------------------
Applying forward operator add=False
 Runs in: 7.381800889968872 seconds
Applying adjoint operator add=False
 Runs in: 7.315475940704346 seconds
Dot products add=False: domain=2.089362e-02 range=2.089362e-02 
Absolute error: 6.245005e-17
Relative error: 2.988953e-15 

Applying forward operator add=True
 Runs in: 7.108118057250977 seconds
Applying adjoint operator add=True
 Runs in: 7.236320972442627 seconds
Dot products add=True: domain=4.178723e-02 range=4.178723e-02 
Absolute error: 1.249001e-16
Relative error: 2.988953e-15 

-------------------------------------------------


## References

Ali Almomin, 2013, Accurate implementation of two-way wave-equation operators, SEP report 149